In [592]:
import json
from web3 import Web3
from web3.middleware import geth_poa_middleware
import pickle
import requests
import pandas as pd
from datetime import datetime
import numpy as np
import math

In [579]:
web3 = Web3(Web3.HTTPProvider("https://bsc-dataseed.binance.org/"))
web3.middleware_onion.inject(geth_poa_middleware, layer=0)
web3.isConnected()

True

In [580]:
#Address = token contract
apikey = 'GKDPH6RRX5YSH1TYPMNHNVKGMB6I9V3I8T'
address = "0xF700D4c708C2be1463E355F337603183D20E0808"

#Lista de topics0

approval_method = '0x8c5be1e5ebec7d5bd14f71427d1e84f3dd0314c0f7b2291e5b200ac8c7c3b925'
transfer_method = '0xddf252ad1be2c89b69c2b068fc378daa952ba7f163c4a11628f55a4df523b3ef'
stake_method = '0xe1fffcc4923d04b559f4d29a8bfc6cda04eb5b0d3c460751c2402c5c5cc9109c'


#Set block
currentblock = web3.eth.blockNumber
endblock = currentblock

#Set topic0
topic0 = approval_method

#Current block
currentblock = web3.eth.blockNumber
endblock = currentblock

#Diferentes topics con su hash
topic_approve = "0x8c5be1e5ebec7d5bd14f71427d1e84f3dd0314c0f7b2291e5b200ac8c7c3b925"
topic_burner = "0x696de425f79f4a40bc6d2122ca50507f0efbeabbff86a84871b7196ab8ea8df7"
topic_transfer = "0xddf252ad1be2c89b69c2b068fc378daa952ba7f163c4a11628f55a4df523b3ef"

claim_contract = "0x739e6e290f9aa6ef3a929950aa47795b1d187229"
stake_contract = "0xdf5c4c1b98a4d045573ab2005d77e87daa58c853"


#Liquidity Pool
pcs = "0x00000000000000000000000072121d60b0e2f01c0fb7fe32ca24021b42165a40"
pcs_address = "0x72121d60b0e2F01c0FB7FE32cA24021b42165A40"

ape = "0xcC3A3Bc1d76Df321f94716E88224638C439267aa"
ape_address = "0x000000000000000000000000cC3A3Bc1d76Df321f94716E88224638C439267aa"

biswap = "0xB4c88543Bd95617Fb61F3d031B8E8E19aF33201E"
biswap_address = "0x000000000000000000000000B4c88543Bd95617Fb61F3d031B8E8E19aF33201E"

#Set block
currentblock = web3.eth.blockNumber
endblock = currentblock

#Set topic0
topic0 = approval_method

In [581]:
#Get ABI

url = "https://api.bscscan.com/api"
params = {
    "module": "contract",
    "action": "getabi",
    "address": address,
    "apikey": apikey
}

response = requests.get(url, params=params)

if response.status_code == 200:
    abi = response.json()["result"]
    print("Abi recibida correctamente")
else:
    print("Error en la solicitud")

Abi recibida correctamente


In [582]:
w3address = Web3.toChecksumAddress(address.lower())
contract = web3.eth.contract(address=w3address, abi=abi)

token = contract.functions.name().call()
symbol = contract.functions.symbol().call()

print(f"Abi del token: {token}, con simbolo: {symbol}, recibida correctamente")

Abi del token: Galactic Quadrant, con simbolo: GQ, recibida correctamente


In [583]:
#Get "Internal Transactions" by Block Range

def get_internal_transactions(endblock, numberofblocks):
    url = "https://api.bscscan.com/api"
    params = {
        "module": "account",
        "action": "txlistinternal",
        "startblock": endblock - numberofblocks,
        "endblock": endblock,
        "page": 1,
        "offset": 10,
        "sort": "asc",
        "apikey": apikey
    }

    response = requests.get(url, params=params)

    if response.status_code == 200:
        transactions = response.json()["result"]
        return transactions
    else:
        return "Error en la solicitud"

#Get Logs by Address sin topic0

def get_logs_loop(address, endblock, blocks):
    if blocks % 1000 != 0:
        raise ValueError("El número de bloques debe ser múltiplo de 1000")
    total_logs = []
    for loop in range(int(blocks/1000)):
        logs = get_logs1000(address,endblock-loop*1000)
        total_logs = total_logs + logs
    return total_logs 


def get_logs1000(address, endblock):
    url = "https://api.bscscan.com/api"
    params = {
        "module": "logs",
        "action": "getLogs",
        "fromBlock": endblock - 1000,
        "toBlock": endblock,
        "address": address,
        "apikey": apikey
    }

    response = requests.get(url, params=params)

    if response.status_code == 200:
        logs = response.json()["result"]
        return logs
    else:
        return "Error en la solicitud"

#Get Logs by Address

def get_logstopic_loop(address, endblock, blocks, topic0):
    if blocks % 1000 != 0:
        raise ValueError("El número de bloques debe ser múltiplo de 1000")
    total_logs = []
    for loop in range(int(blocks/1000)):
        logs = get_logs1000(address,endblock-loop*1000)
        total_logs = total_logs + logs
    return total_logs 


def get_logs1000topic(address, endblock, topic0):
    url = "https://api.bscscan.com/api"
    params = {
        "module": "logs",
        "action": "getLogs",
        "fromBlock": endblock - 1000,
        "toBlock": endblock,
        "topic0": topic0,
        "address": address,
        "apikey": apikey
    }

    response = requests.get(url, params=params)

    if response.status_code == 200:
        logs = response.json()["result"]
        return logs
    else:
        return "Error en la solicitud"


#Get Logs by Address and From

def get_logstopic0_1_loop(address, endblock, blocks, topic0, topic1):
    if blocks % 1000 != 0:
        raise ValueError("El número de bloques debe ser múltiplo de 1000")
    total_logs = []
    for loop in range(int(blocks/1000)):
        logs = get_logs1000(address,endblock-loop*1000)
        total_logs = total_logs + logs
    return total_logs 


def get_logs1000topic0_1(address, endblock, topic0, topic1):
    url = "https://api.bscscan.com/api"
    params = {
        "module": "logs",
        "action": "getLogs",
        "fromBlock": endblock - 1000,
        "toBlock": endblock,
        "topic0": topic0,
        "topic0_1": "and",
        "topic1": topic1,                
        "address": address,
        "apikey": apikey
    }

    response = requests.get(url, params=params)

    if response.status_code == 200:
        logs = response.json()["result"]
        return logs
    else:
        return "Error en la solicitud"

#Get a list of 'BEP-20 Token Transfer Events' by wallet

def get_token_transfers(wallet, address):
    url = "https://api.bscscan.com/api"
    params = {
        "module": "account",
        "action": "tokentx",
        "contractaddress": address,
        "address": wallet,
        "page": 1,
        "offset": 1000,
        "startblock": 0,
        "endblock": 999999999,
        "sort": "asc",
        "apikey": apikey,
    }

    response = requests.get(url, params=params)

    if response.status_code == 200:
        transactions = response.json()["result"]
        return transactions
    else:
        return "Error in the request"


In [600]:
## Extractor de Events/topics0
total_logs = get_logs_loop(address,endblock,100000)


#Save hash in a list
total_hash = []
for i in range(len(total_logs)):
    event_hash = total_logs[i]['topics'][0]
    total_hash.append(event_hash)

unique_event_hash = list(set(total_hash))


#Cantidad de logs extraidos
print(f"Se han obtenido {len(total_logs)} logs, en los cuales hay {len(unique_event_hash)} event hash unicos.")
print(unique_event_hash)


#Guarda los nuevos event hash
logs_unique = []
for value in unique_event_hash:
    if value not in logs_unique:
        logs_unique.append(value)

print(f"Actualmente hay almacenados {len(logs_unique)} event hash unicos que utilizaremos para reconocer los diferentes methods de topic0")


Se han obtenido 5891 logs, en los cuales hay 2 event hash unicos.
['0x8c5be1e5ebec7d5bd14f71427d1e84f3dd0314c0f7b2291e5b200ac8c7c3b925', '0xddf252ad1be2c89b69c2b068fc378daa952ba7f163c4a11628f55a4df523b3ef']
Actualmente hay almacenados 2 event hash unicos que utilizaremos para reconocer los diferentes methods de topic0


In [614]:
#Un dia son aprox 28200 bloques, queremos calcular el finde pasado -5/-7 dias
currentblock = web3.eth.blockNumber
endblock = currentblock - (28200*5)
blocks = 28200 * 2.5
blocks = math.ceil(blocks / 1000.0) * 1000 #Multiplos de 1000

#Get data
logs_weekend = get_logs_loop(address, endblock, blocks)
token_tx = pd.DataFrame(logs_weekend)
token_tx.head()

,address,topics,data,blockNumber,blockHash,timeStamp,gasPrice,gasUsed,logIndex,transactionHash,transactionIndex
0,0xf700d4c708c2be1463e355f337603183d20e0808,"[0xddf252ad1be2c89b69c2b068fc378daa952ba7f163c4a11628f55a4df523b3ef, 0x0000000000000000000000005...",0x00000000000000000000000000000000000000000000001e757b3db8d9fc6000,0x180bd71,0xacc710675a05d7e8635c722391b413bfad5efe0917d0160c625c222ab724123e,0x63d6cc1c,0x12a05f200,0x2eae8,0xff,0xcca5986872f7f9b1323d75c0acb7487442da9eafcac8bf448bad8e205b743e2c,0x37
1,0xf700d4c708c2be1463e355f337603183d20e0808,"[0xddf252ad1be2c89b69c2b068fc378daa952ba7f163c4a11628f55a4df523b3ef, 0x000000000000000000000000e...",0x0000000000000000000000000000000000000000000043c866768a5ad5df0400,0x180bd72,0xe843db1a69c472f71e64d40c54b1d68c10a6549c247c4f6a151c570fdb8f7b7d,0x63d6cc1f,0x1dcd65000,0xa2cc,0x4,0x94e67e17f710190c8c1c6b5a045e0e9dc28583bf34310f0d2f4b9f0f2b322a77,0x4
2,0xf700d4c708c2be1463e355f337603183d20e0808,"[0xddf252ad1be2c89b69c2b068fc378daa952ba7f163c4a11628f55a4df523b3ef, 0x000000000000000000000000b...",0x000000000000000000000000000000000000000000003647eb532de13b8ef61a,0x180bd79,0x8cf937e2e97d79b942577f948db5f04ef81a3cda8d2309338b668eeadfaf2271,0x63d6cc34,0x12a05f200,0x6840,0x5a,0x5aca24239d1b3e3d02483dd7ec687376a84fa66d8fe219e837de5d46accdfeb1,0x28
3,0xf700d4c708c2be1463e355f337603183d20e0808,"[0xddf252ad1be2c89b69c2b068fc378daa952ba7f163c4a11628f55a4df523b3ef, 0x0000000000000000000000007...",0x000000000000000000000000000000000000000000003f870857a3e0e3800000,0x180bd7f,0xbcb09c0c4cb5ae21f6be62b9a1b1f30b08951904fc97935ac1f908fb83bcacaf,0x63d6cc46,0x12a05f200,0x3541e,0xfe,0x1d91b5764042d08e84c5bb8d385ad4cf383ed6efe31106357563f242abe0bb31,0x43
4,0xf700d4c708c2be1463e355f337603183d20e0808,"[0xddf252ad1be2c89b69c2b068fc378daa952ba7f163c4a11628f55a4df523b3ef, 0x000000000000000000000000c...",0x00000000000000000000000000000000000000000000050f180b690a75d84c3e,0x180bd87,0xf82e1269cca01c2ea8bf6e110ed8cca4ba8975b71fc40ff9652dd45682d1900d,0x63d6cc5e,0x3b9aca01,0x289d2,0xf9,0x5899e0fb871646d431b0b0adb63670de6c6725819f13132e0a19eccf37c93051,0x46


## TOKEN LOG CLEANING

In [615]:
token_tx.head()

,address,topics,data,blockNumber,blockHash,timeStamp,gasPrice,gasUsed,logIndex,transactionHash,transactionIndex
0,0xf700d4c708c2be1463e355f337603183d20e0808,"[0xddf252ad1be2c89b69c2b068fc378daa952ba7f163c4a11628f55a4df523b3ef, 0x0000000000000000000000005...",0x00000000000000000000000000000000000000000000001e757b3db8d9fc6000,0x180bd71,0xacc710675a05d7e8635c722391b413bfad5efe0917d0160c625c222ab724123e,0x63d6cc1c,0x12a05f200,0x2eae8,0xff,0xcca5986872f7f9b1323d75c0acb7487442da9eafcac8bf448bad8e205b743e2c,0x37
1,0xf700d4c708c2be1463e355f337603183d20e0808,"[0xddf252ad1be2c89b69c2b068fc378daa952ba7f163c4a11628f55a4df523b3ef, 0x000000000000000000000000e...",0x0000000000000000000000000000000000000000000043c866768a5ad5df0400,0x180bd72,0xe843db1a69c472f71e64d40c54b1d68c10a6549c247c4f6a151c570fdb8f7b7d,0x63d6cc1f,0x1dcd65000,0xa2cc,0x4,0x94e67e17f710190c8c1c6b5a045e0e9dc28583bf34310f0d2f4b9f0f2b322a77,0x4
2,0xf700d4c708c2be1463e355f337603183d20e0808,"[0xddf252ad1be2c89b69c2b068fc378daa952ba7f163c4a11628f55a4df523b3ef, 0x000000000000000000000000b...",0x000000000000000000000000000000000000000000003647eb532de13b8ef61a,0x180bd79,0x8cf937e2e97d79b942577f948db5f04ef81a3cda8d2309338b668eeadfaf2271,0x63d6cc34,0x12a05f200,0x6840,0x5a,0x5aca24239d1b3e3d02483dd7ec687376a84fa66d8fe219e837de5d46accdfeb1,0x28
3,0xf700d4c708c2be1463e355f337603183d20e0808,"[0xddf252ad1be2c89b69c2b068fc378daa952ba7f163c4a11628f55a4df523b3ef, 0x0000000000000000000000007...",0x000000000000000000000000000000000000000000003f870857a3e0e3800000,0x180bd7f,0xbcb09c0c4cb5ae21f6be62b9a1b1f30b08951904fc97935ac1f908fb83bcacaf,0x63d6cc46,0x12a05f200,0x3541e,0xfe,0x1d91b5764042d08e84c5bb8d385ad4cf383ed6efe31106357563f242abe0bb31,0x43
4,0xf700d4c708c2be1463e355f337603183d20e0808,"[0xddf252ad1be2c89b69c2b068fc378daa952ba7f163c4a11628f55a4df523b3ef, 0x000000000000000000000000c...",0x00000000000000000000000000000000000000000000050f180b690a75d84c3e,0x180bd87,0xf82e1269cca01c2ea8bf6e110ed8cca4ba8975b71fc40ff9652dd45682d1900d,0x63d6cc5e,0x3b9aca01,0x289d2,0xf9,0x5899e0fb871646d431b0b0adb63670de6c6725819f13132e0a19eccf37c93051,0x46


In [619]:
#Set Wallet
wallet = "0xdb5de412A2bF5eD7625288558bda49A39C20Bfe8"

#Data Clean
# token_tx['timeStamp'] = token_tx['timeStamp'].apply(lambda x: int(x, 16))
# token_tx['timeStamp'] = token_tx['timeStamp'].apply(lambda x: datetime.fromtimestamp(x))

df_topics = pd.DataFrame(token_tx['topics'].tolist(), columns=['method', 'from', 'to'])
token_tx = pd.concat([token_tx, df_topics], axis=1)

token_tx = token_tx.drop(
    columns=[
        "Topics",
        "Address",
        "data",
        "blockNumber",
        "blockHash",
        "gasPrice",
        "gasUsed",
        "logIndex",
        "gasUsed",
        "transactionIndex",
    ],
    axis=1,
)

token_tx.sort_values(by='timeStamp', ascending=True, inplace=True)

#Add balance 
# token_tx['move_balance'] = np.where(token_tx['to'] == wallet.lower(), token_tx['value'], -token_tx['value'])
# balance = token_tx['move_balance'].sum()

#Staking Check & Balance
# token_tx['staking'] = ((token_tx['to'] == stake_contract) | (token_tx['from'] == stake_contract)).astype(int)


# token_tx['stake_balance'] = 0
# for index, row in token_tx.iterrows():
#     if row['staking'] == 1:
#         if row['to'] == wallet.lower():
#             token_tx.at[index, 'stake_balance'] = row['value']
#         else:
#             token_tx.at[index, 'stake_balance'] = -row['value']




KeyError: "['Topics' 'Address' 'data' 'blockNumber' 'blockHash' 'gasPrice' 'gasUsed'\n 'logIndex' 'gasUsed' 'transactionIndex'] not found in axis"

In [ ]:
#Set Wallet
wallet = "0xdb5de412A2bF5eD7625288558bda49A39C20Bfe8"

#Data Clean
token_tx['timeStamp'] = token_tx['timeStamp'].astype(int)
token_tx['timeStamp'] = token_tx['timeStamp'].apply(lambda x: datetime.fromtimestamp(x))

df_topics = pd.DataFrame(token_tx['topics'].tolist(), columns=['method', 'from', 'to'])
token_tx = pd.concat([token_tx, df_topics], axis=1)

token_tx['mont'] = round(token_tx['value'].astype(float)/1000000000000000000, 2)

token_tx = token_tx.drop(
    columns=[
        "blockNumber",
        "nonce",
        "contractAddress",
        "tokenName",
        "transactionIndex",
        "gas",
        "gasPrice",
        "gasUsed",
        "cumulativeGasUsed",
        "input",
        "confirmations",
        "blockHash",
        "value",
        "tokenDecimal",
        "Topics",
    ],
    axis=1,
)

# token_tx.rename(columns={'mont' : 'value'}, inplace=True)

token_tx.sort_values(by='timeStamp', ascending=True, inplace=True)

#Add balance 
# token_tx['move_balance'] = np.where(token_tx['to'] == wallet.lower(), token_tx['value'], -token_tx['value'])
# balance = token_tx['move_balance'].sum()

#Staking Check & Balance
# token_tx['staking'] = ((token_tx['to'] == stake_contract) | (token_tx['from'] == stake_contract)).astype(int)


# token_tx['stake_balance'] = 0
# for index, row in token_tx.iterrows():
#     if row['staking'] == 1:
#         if row['to'] == wallet.lower():
#             token_tx.at[index, 'stake_balance'] = row['value']
#         else:
#             token_tx.at[index, 'stake_balance'] = -row['value']




In [589]:

#Balance Staked
print(f"Actualmente tiene {token_tx['stake_balance'].sum()} tokens stakeados")

#Tokens holds
move_balance = token_tx.loc[token_tx['staking'] == 0, 'move_balance']
print(f"y holdea {move_balance.sum() + token_tx['stake_balance'].sum()} tokens.")

Actualmente tiene 18256.119999999733 tokens stakeados
y holdea 314.03999999930966 tokens.


In [575]:
token_tx.head()

,timeStamp,hash,from,to,tokenSymbol,value,move_balance,staking,stake_balance
0,2022-08-17 17:34:51,0x87a77431ebc835b825988b18d17565b7e1baa114dc82a4d9a17a6dc18c30cddb,0x72121d60b0e2f01c0fb7fe32ca24021b42165a40,0xdb5de412a2bf5ed7625288558bda49a39c20bfe8,GQ,80690.37,80690.37,0,0.00
1,2022-08-18 17:53:07,0xe209c0cd15c80c026a92276bd6ef3901c3f365f3b379465f0b55d8b23c9f0dd0,0xdb5de412a2bf5ed7625288558bda49a39c20bfe8,0xdf5c4c1b98a4d045573ab2005d77e87daa58c853,GQ,80690.37,-80690.37,1,-80690.37
2,2022-08-31 14:27:18,0x938f4fe05038827f0013531fdc91bca8121615da8b98e187da6be6a33a41607d,0xdf5c4c1b98a4d045573ab2005d77e87daa58c853,0xdb5de412a2bf5ed7625288558bda49a39c20bfe8,GQ,135.62,135.62,1,135.62
3,2022-08-31 14:27:48,0x466b335eca3006b9a9b53f189095e5c8ca520754c4afa625ca47ad3db1b6b123,0xdf5c4c1b98a4d045573ab2005d77e87daa58c853,0xdb5de412a2bf5ed7625288558bda49a39c20bfe8,GQ,0.00,0.00,1,0.00
4,2022-08-31 14:27:48,0x466b335eca3006b9a9b53f189095e5c8ca520754c4afa625ca47ad3db1b6b123,0xdb5de412a2bf5ed7625288558bda49a39c20bfe8,0xdf5c4c1b98a4d045573ab2005d77e87daa58c853,GQ,135.62,-135.62,1,-135.62


In [572]:
token_tx

,timeStamp,hash,from,to,tokenSymbol,value,move_balance,staking,stake_balance
0,2022-08-17 17:34:51,0x87a77431ebc835b825988b18d17565b7e1baa114dc82a4d9a17a6dc18c30cddb,0x72121d60b0e2f01c0fb7fe32ca24021b42165a40,0xdb5de412a2bf5ed7625288558bda49a39c20bfe8,GQ,80690.37,-80690.37,0,0.00
1,2022-08-18 17:53:07,0xe209c0cd15c80c026a92276bd6ef3901c3f365f3b379465f0b55d8b23c9f0dd0,0xdb5de412a2bf5ed7625288558bda49a39c20bfe8,0xdf5c4c1b98a4d045573ab2005d77e87daa58c853,GQ,80690.37,-80690.37,1,-80690.37
2,2022-08-31 14:27:18,0x938f4fe05038827f0013531fdc91bca8121615da8b98e187da6be6a33a41607d,0xdf5c4c1b98a4d045573ab2005d77e87daa58c853,0xdb5de412a2bf5ed7625288558bda49a39c20bfe8,GQ,135.62,-135.62,1,135.62
3,2022-08-31 14:27:48,0x466b335eca3006b9a9b53f189095e5c8ca520754c4afa625ca47ad3db1b6b123,0xdf5c4c1b98a4d045573ab2005d77e87daa58c853,0xdb5de412a2bf5ed7625288558bda49a39c20bfe8,GQ,0.00,-0.00,1,0.00
4,2022-08-31 14:27:48,0x466b335eca3006b9a9b53f189095e5c8ca520754c4afa625ca47ad3db1b6b123,0xdb5de412a2bf5ed7625288558bda49a39c20bfe8,0xdf5c4c1b98a4d045573ab2005d77e87daa58c853,GQ,135.62,-135.62,1,-135.62
5,2022-09-01 16:54:47,0x148e9606449902fe24133b32088e13af42088c98b77ca0fc23fca940ee59588d,0x72121d60b0e2f01c0fb7fe32ca24021b42165a40,0xdb5de412a2bf5ed7625288558bda49a39c20bfe8,GQ,198570.04,-198570.04,0,0.00
6,2022-09-01 16:56:53,0x46bd24a027f3e514e515465c3dc0c040d4f9e2e46e2f1b579c8f7b808d4d8b2f,0xdf5c4c1b98a4d045573ab2005d77e87daa58c853,0xdb5de412a2bf5ed7625288558bda49a39c20bfe8,GQ,11.85,-11.85,1,11.85
7,2022-09-01 16:56:53,0x46bd24a027f3e514e515465c3dc0c040d4f9e2e46e2f1b579c8f7b808d4d8b2f,0xdb5de412a2bf5ed7625288558bda49a39c20bfe8,0xdf5c4c1b98a4d045573ab2005d77e87daa58c853,GQ,198570.04,-198570.04,1,-198570.04
8,2022-09-08 10:14:17,0x77f01cc31d0770b891be275762c8f4e8899861824e5d1f3b2472fc2e7c85259a,0x72121d60b0e2f01c0fb7fe32ca24021b42165a40,0xdb5de412a2bf5ed7625288558bda49a39c20bfe8,GQ,355542.31,-355542.31,0,0.00
9,2022-09-08 10:32:05,0x420b7963f61d06020765c410fa3d9fa279e38524b6f784928e5f706fffe32ee9,0xdf5c4c1b98a4d045573ab2005d77e87daa58c853,0xdb5de412a2bf5ed7625288558bda49a39c20bfe8,GQ,241.44,-241.44,1,241.44
